In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame
from pandas import Series
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
font_path = 'AppleGothic.ttf'
fontprop = fm.FontProperties(fname=font_path, size=18)
font_name = fm.FontProperties(fname=font_path).get_name()

# 단위근 검정 하나씩

In [13]:
food_waste = pd.read_csv("01_음식물쓰레기_FOOD_WASTE_210811_update.CSV", encoding="cp949")
card_cluster = pd.read_csv("card_cluster.csv")

In [234]:
card = card_cluster.copy()

In [180]:
def get_year_month(x):
    return x[0:7]

In [181]:
card.rename(columns={'cluster':'emd_cluster'}, inplace=True)
card['year_month'] = card['base_date'].apply(lambda x: get_year_month(x))
card= card.drop(['base_date'], axis=1, inplace=False)
card.rename(columns={'cluster':'emd_cluster'}, inplace=True)

card_1 = card.groupby(['year_month','emd_cluster','card_cluster']).sum().reset_index()
card_1= card_1.drop(['Unnamed: 0','use_cnt'], axis=1, inplace=False)
card_1['year_month'] = pd.to_datetime(card_1['year_month'])
card_1.set_index('year_month',inplace=True)

In [182]:
card_1

,emd_cluster,card_cluster,use_amt
year_month,,,
2018-01-01,1.0,1,3667354305
2018-01-01,1.0,2,9160831148
2018-01-01,1.0,3,7766426029
2018-01-01,2.0,1,8117543273
2018-01-01,2.0,2,1043206866
...,...,...,...
2021-06-01,8.0,2,6915463393
2021-06-01,8.0,3,2373658
2021-06-01,9.0,1,9193643853


In [186]:
df = card_1[(card_1['emd_cluster']==1)& (card_1['card_cluster']==1)]

In [177]:
from statsmodels.tsa.stattools import adfuller

def adf_test(df):
    result = adfuller(df.values)
    print('ADF Statistics: %f' % result[0])
    print('p-value: %f' %result[1])
    print('Critical values:')
    for key, value in result[4].items():
        print('\t%s: %.3f' %(key, value))

In [190]:
adf_test(df['use_amt']) #안정적이지 않음

ADF Statistics: 1.156904
p-value: 0.995674
Critical values:
	1%: -3.661
	5%: -2.961
	10%: -2.619


# 단위근 검정 한번에

In [239]:
card_cluster = pd.read_csv("card_cluster.csv")

In [240]:
card = card_cluster.copy()

In [241]:
def get_year_month(x):
    return x[0:7]

In [242]:
card.rename(columns={'cluster':'emd_cluster'}, inplace=True)
card['year_month'] = card['base_date'].apply(lambda x: get_year_month(x))
card= card.drop(['base_date'], axis=1, inplace=False)
card.rename(columns={'cluster':'emd_cluster'}, inplace=True)

card_1 = card.groupby(['year_month','emd_cluster','card_cluster']).sum().reset_index()
card_1= card_1.drop(['Unnamed: 0','use_cnt'], axis=1, inplace=False)
card_1.set_index('year_month',inplace=True)

In [243]:
from statsmodels.tsa.stattools import adfuller

def adf_test(df):
    result = adfuller(df.values)
    print('ADF Statistics: %f' % result[0])
    print('p-value: %f' %result[1])
    print('Critical values:')
    for key, value in result[4].items():
        print('\t%s: %.3f' %(key, value))

In [245]:
card_1

,emd_cluster,card_cluster,use_amt
year_month,,,
2018-01,1.0,1,3667354305
2018-01,1.0,2,9160831148
2018-01,1.0,3,7766426029
2018-01,2.0,1,8117543273
2018-01,2.0,2,1043206866
...,...,...,...
2021-06,8.0,2,6915463393
2021-06,8.0,3,2373658
2021-06,9.0,1,9193643853


In [251]:
from statsmodels.tsa.stattools import adfuller
temp_cluster = []
temp_diff = []
temp_log = []
temp_log_diff = []

for i in tqdm(range(1,10)):
    temp = card_1[card_1['emd_cluster']==i]
    card_list = set(temp['card_cluster'])
    
    for j in card_list:
        temp2= temp[temp['card_cluster']==j]
        
        temp_diff_result = adfuller(temp2['use_amt'].diff()[1:].values)
        temp_result = adfuller(np.log(temp2['use_amt']).values)
        temp_result_diff =  adfuller(np.log(temp2['use_amt']).diff()[1:].values)
    
        
        print("diff")
        print("p-values = ", round(temp_diff_result[1],3), " : emd_cluster", i, ' card_cluster', j)
        print("log")
        print("p-values = ", round(temp_result[1],3), " : emd_cluster", i, ' card_cluster', j)
        print("log.diff")
        print("p-values = ", round(temp_result_diff[1],3), " : emd_cluster", i, ' card_cluster', j)
        temp_cluster.append(str(i) + '-' + str(j))
        
        if (temp_diff_result[1] < 0.05):
            temp_diff.append(temp_diff_result[1])
        else:
            temp_diff.append(np.nan)
        if (temp_result[1] < 0.05):
            temp_log.append(temp_result[1])
        else:
            temp_log.append(np.nan)
        if (temp_result_diff[1] < 0.05):
            temp_log_diff.append(temp_result_diff[1])
        else:
            temp_log_diff.append(np.nan)

 22%|██▏       | 2/9 [00:00<00:00, 16.16it/s]

diff
p-values =  0.059  : emd_cluster 1  card_cluster 1
log
p-values =  0.992  : emd_cluster 1  card_cluster 1
log.diff
p-values =  0.032  : emd_cluster 1  card_cluster 1
diff
p-values =  0.017  : emd_cluster 1  card_cluster 2
log
p-values =  0.028  : emd_cluster 1  card_cluster 2
log.diff
p-values =  0.02  : emd_cluster 1  card_cluster 2
diff
p-values =  0.023  : emd_cluster 1  card_cluster 3
log
p-values =  0.0  : emd_cluster 1  card_cluster 3
log.diff
p-values =  0.027  : emd_cluster 1  card_cluster 3
diff
p-values =  0.0  : emd_cluster 2  card_cluster 1
log
p-values =  0.016  : emd_cluster 2  card_cluster 1
log.diff
p-values =  0.0  : emd_cluster 2  card_cluster 1
diff
p-values =  0.0  : emd_cluster 2  card_cluster 2
log
p-values =  0.915  : emd_cluster 2  card_cluster 2
log.diff
p-values =  0.0  : emd_cluster 2  card_cluster 2
diff
p-values =  0.047  : emd_cluster 2  card_cluster 3
log
p-values =  0.0  : emd_cluster 2  card_cluster 3
log.diff
p-values =  0.061  : emd_cluster 2  ca

 67%|██████▋   | 6/9 [00:00<00:00, 16.30it/s]

diff
p-values =  0.0  : emd_cluster 4  card_cluster 2
log
p-values =  0.004  : emd_cluster 4  card_cluster 2
log.diff
p-values =  0.0  : emd_cluster 4  card_cluster 2
diff
p-values =  0.0  : emd_cluster 4  card_cluster 3
log
p-values =  0.004  : emd_cluster 4  card_cluster 3
log.diff
p-values =  0.0  : emd_cluster 4  card_cluster 3
diff
p-values =  0.0  : emd_cluster 5  card_cluster 1
log
p-values =  0.002  : emd_cluster 5  card_cluster 1
log.diff
p-values =  0.0  : emd_cluster 5  card_cluster 1
diff
p-values =  0.0  : emd_cluster 5  card_cluster 2
log
p-values =  0.312  : emd_cluster 5  card_cluster 2
log.diff
p-values =  0.0  : emd_cluster 5  card_cluster 2
diff
p-values =  0.0  : emd_cluster 5  card_cluster 3
log
p-values =  0.994  : emd_cluster 5  card_cluster 3
log.diff
p-values =  0.001  : emd_cluster 5  card_cluster 3
diff
p-values =  0.016  : emd_cluster 6  card_cluster 1
log
p-values =  0.985  : emd_cluster 6  card_cluster 1
log.diff
p-values =  0.008  : emd_cluster 6  card_cl

100%|██████████| 9/9 [00:00<00:00, 16.36it/s]

diff
p-values =  0.556  : emd_cluster 7  card_cluster 3
log
p-values =  0.848  : emd_cluster 7  card_cluster 3
log.diff
p-values =  0.136  : emd_cluster 7  card_cluster 3
diff
p-values =  0.0  : emd_cluster 8  card_cluster 1
log
p-values =  0.805  : emd_cluster 8  card_cluster 1
log.diff
p-values =  0.0  : emd_cluster 8  card_cluster 1
diff
p-values =  0.003  : emd_cluster 8  card_cluster 2
log
p-values =  0.006  : emd_cluster 8  card_cluster 2
log.diff
p-values =  0.002  : emd_cluster 8  card_cluster 2
diff
p-values =  0.0  : emd_cluster 8  card_cluster 3
log
p-values =  0.607  : emd_cluster 8  card_cluster 3
log.diff
p-values =  0.0  : emd_cluster 8  card_cluster 3
diff
p-values =  0.016  : emd_cluster 9  card_cluster 1
log
p-values =  0.006  : emd_cluster 9  card_cluster 1
log.diff
p-values =  0.016  : emd_cluster 9  card_cluster 1
diff
p-values =  0.001  : emd_cluster 9  card_cluster 2
log
p-values =  0.933  : emd_cluster 9  card_cluster 2
log.diff
p-values =  0.001  : emd_cluster 

In [258]:
method_card = pd.DataFrame()
from statsmodels.tsa.stattools import adfuller
temp_cluster = []
temp_diff = []
temp_log = []
temp_log_diff = []
for i in tqdm(range(1,10)):
    temp = card_1[card_1['emd_cluster']==i]
    card_list = set(temp['card_cluster'])
    
    for j in card_list:
        temp2= temp[temp['card_cluster']==j]
        
        temp_diff_result = adfuller(temp2['use_amt'].diff()[1:].values)
        temp_result = adfuller(np.log(temp2['use_amt']).values)
        temp_result_diff =  adfuller(np.log(temp2['use_amt']).diff()[1:].values)

        print("diff")
        print("p-values = ", round(temp_diff_result[1],3), " : emd_cluster", i, ' age_cluster', j)
        print("log")
        print("p-values = ", round(temp_result[1],3), " : emd_cluster", i, ' age_cluster', j)
        print("log.diff")
        print("p-values = ", round(temp_result_diff[1],3), " : emd_cluster", i, ' age_cluster', j)
        temp_cluster.append(str(i) + '-' + str(j))
        temp_diff.append(temp_diff_result[1])
        temp_log.append(temp_result[1])
        temp_log_diff.append(temp_result_diff[1])
        
method_card['cluster'] = temp_cluster
method_card['diff'] = temp_diff
method_card['log'] = temp_log
method_card['log_diff'] = temp_log_diff
method_card['method'] = method_card[['diff', 'log', 'log_diff']].idxmin(axis=1)
method_card

 22%|██▏       | 2/9 [00:00<00:00, 15.96it/s]

diff
p-values =  0.059  : emd_cluster 1  age_cluster 1
log
p-values =  0.992  : emd_cluster 1  age_cluster 1
log.diff
p-values =  0.032  : emd_cluster 1  age_cluster 1
diff
p-values =  0.017  : emd_cluster 1  age_cluster 2
log
p-values =  0.028  : emd_cluster 1  age_cluster 2
log.diff
p-values =  0.02  : emd_cluster 1  age_cluster 2
diff
p-values =  0.023  : emd_cluster 1  age_cluster 3
log
p-values =  0.0  : emd_cluster 1  age_cluster 3
log.diff
p-values =  0.027  : emd_cluster 1  age_cluster 3
diff
p-values =  0.0  : emd_cluster 2  age_cluster 1
log
p-values =  0.016  : emd_cluster 2  age_cluster 1
log.diff
p-values =  0.0  : emd_cluster 2  age_cluster 1
diff
p-values =  0.0  : emd_cluster 2  age_cluster 2
log
p-values =  0.915  : emd_cluster 2  age_cluster 2
log.diff
p-values =  0.0  : emd_cluster 2  age_cluster 2
diff
p-values =  0.047  : emd_cluster 2  age_cluster 3
log
p-values =  0.0  : emd_cluster 2  age_cluster 3
log.diff
p-values =  0.061  : emd_cluster 2  age_cluster 3
diff


 67%|██████▋   | 6/9 [00:00<00:00, 16.16it/s]

diff
p-values =  0.0  : emd_cluster 4  age_cluster 2
log
p-values =  0.004  : emd_cluster 4  age_cluster 2
log.diff
p-values =  0.0  : emd_cluster 4  age_cluster 2
diff
p-values =  0.0  : emd_cluster 4  age_cluster 3
log
p-values =  0.004  : emd_cluster 4  age_cluster 3
log.diff
p-values =  0.0  : emd_cluster 4  age_cluster 3
diff
p-values =  0.0  : emd_cluster 5  age_cluster 1
log
p-values =  0.002  : emd_cluster 5  age_cluster 1
log.diff
p-values =  0.0  : emd_cluster 5  age_cluster 1
diff
p-values =  0.0  : emd_cluster 5  age_cluster 2
log
p-values =  0.312  : emd_cluster 5  age_cluster 2
log.diff
p-values =  0.0  : emd_cluster 5  age_cluster 2
diff
p-values =  0.0  : emd_cluster 5  age_cluster 3
log
p-values =  0.994  : emd_cluster 5  age_cluster 3
log.diff
p-values =  0.001  : emd_cluster 5  age_cluster 3
diff
p-values =  0.016  : emd_cluster 6  age_cluster 1
log
p-values =  0.985  : emd_cluster 6  age_cluster 1
log.diff
p-values =  0.008  : emd_cluster 6  age_cluster 1
diff
p-val

100%|██████████| 9/9 [00:00<00:00, 16.25it/s]

diff
p-values =  0.556  : emd_cluster 7  age_cluster 3
log
p-values =  0.848  : emd_cluster 7  age_cluster 3
log.diff
p-values =  0.136  : emd_cluster 7  age_cluster 3
diff
p-values =  0.0  : emd_cluster 8  age_cluster 1
log
p-values =  0.805  : emd_cluster 8  age_cluster 1
log.diff
p-values =  0.0  : emd_cluster 8  age_cluster 1
diff
p-values =  0.003  : emd_cluster 8  age_cluster 2
log
p-values =  0.006  : emd_cluster 8  age_cluster 2
log.diff
p-values =  0.002  : emd_cluster 8  age_cluster 2
diff
p-values =  0.0  : emd_cluster 8  age_cluster 3
log
p-values =  0.607  : emd_cluster 8  age_cluster 3
log.diff
p-values =  0.0  : emd_cluster 8  age_cluster 3
diff
p-values =  0.016  : emd_cluster 9  age_cluster 1
log
p-values =  0.006  : emd_cluster 9  age_cluster 1
log.diff
p-values =  0.016  : emd_cluster 9  age_cluster 1
diff
p-values =  0.001  : emd_cluster 9  age_cluster 2
log
p-values =  0.933  : emd_cluster 9  age_cluster 2
log.diff
p-values =  0.001  : emd_cluster 9  age_cluster 2


,cluster,diff,log,log_diff,method
0,1-1,5.895797e-02,9.924947e-01,3.234714e-02,log_diff
1,1-2,1.674230e-02,2.785654e-02,1.991698e-02,diff
2,1-3,2.336337e-02,2.067938e-04,2.664627e-02,log
3,2-1,2.156804e-04,1.605224e-02,3.904277e-04,diff
4,2-2,6.268536e-09,9.149116e-01,2.768777e-07,diff
5,2-3,4.659474e-02,2.593174e-07,6.109552e-02,log
6,3-1,1.599123e-02,5.414398e-03,1.495234e-02,log
7,3-2,7.061176e-03,9.260183e-01,5.559721e-03,log_diff
8,3-3,7.174089e-01,3.955467e-01,1.637126e-07,log_diff
9,4-1,1.101549e-09,8.916208e-01,2.975168e-10,log_diff


In [259]:
method_card.to_csv("card_ADF.csv", index=False)

In [198]:
#차분
for i in tqdm(range(1,10)):
    temp = card_1[card_1['emd_cluster']==i]
    card_list = set(temp['card_cluster'])
    for j in card_list:
        temp2= temp[temp['card_cluster']==j]
        temp3 = temp2['use_amt'].diff()
        temp3.dropna(inplace=True)
        result = adfuller(temp3.values)
        if result[1] >= 0.05:
            print("p-values = ", round(result[1],3), ': emd_cluster',i, ', card_cluster', j)

100%|██████████| 9/9 [00:00<00:00, 46.55it/s]

p-values =  0.059 : emd_cluster 1 , card_cluster 1
p-values =  0.717 : emd_cluster 3 , card_cluster 3
p-values =  0.556 : emd_cluster 7 , card_cluster 3


In [232]:
#로그
for i in tqdm(range(1,10)):
    temp = card_1[card_1['emd_cluster']==i]
    card_list = set(temp['card_cluster'])
    for j in card_list:
        temp2= temp[temp['card_cluster']==j]
        temp3 = np.log(temp2['use_amt'])
        temp3.dropna(inplace=True)
        result = adfuller(temp3.values)
        if result[1] >= 0.05:
            print("p-values = ", round(result[1],3), ': emd_cluster',i, ', card_cluster', j)

 33%|███▎      | 3/9 [00:00<00:00, 28.00it/s]

p-values =  0.792 : emd_cluster 1 , card_cluster 1
p-values =  0.083 : emd_cluster 1 , card_cluster 2
p-values =  0.915 : emd_cluster 2 , card_cluster 2
p-values =  0.122 : emd_cluster 3 , card_cluster 2
p-values =  0.596 : emd_cluster 3 , card_cluster 3
p-values =  0.121 : emd_cluster 4 , card_cluster 1
p-values =  0.081 : emd_cluster 4 , card_cluster 2
p-values =  0.272 : emd_cluster 5 , card_cluster 2
p-values =  0.741 : emd_cluster 6 , card_cluster 1


100%|██████████| 9/9 [00:00<00:00, 28.31it/s]

p-values =  0.125 : emd_cluster 7 , card_cluster 2
p-values =  0.673 : emd_cluster 7 , card_cluster 3
p-values =  0.913 : emd_cluster 8 , card_cluster 1
p-values =  0.543 : emd_cluster 8 , card_cluster 3
p-values =  0.058 : emd_cluster 9 , card_cluster 2
p-values =  0.84 : emd_cluster 9 , card_cluster 3


In [199]:
# 로그&차분
for i in tqdm(range(1,10)):
    temp = card_1[card_1['emd_cluster']==i]
    card_list = set(temp['card_cluster'])
    for j in card_list:
        temp2= temp[temp['card_cluster']==j]
        temp3 = np.log(temp2['use_amt']).diff()
        temp3.dropna(inplace=True)
        result = adfuller(temp3.values)
        if result[1] >= 0.05:
            print("p-values = ", round(result[1],3), ': emd_cluster',i, ', card_cluster', j)

100%|██████████| 9/9 [00:00<00:00, 45.33it/s]

p-values =  0.061 : emd_cluster 2 , card_cluster 3
p-values =  0.136 : emd_cluster 7 , card_cluster 3


# 공적분 검정

In [204]:
food_waste = pd.read_csv("01_음식물쓰레기_FOOD_WASTE_210811_update.CSV", encoding='cp949')
card_cluster = pd.read_csv("card_cluster.csv")

card = card_cluster.copy()
fw= food_waste.copy()

def get_year_month(x):
    return x[0:7]

fw['year_month'] = fw['base_date'].apply(lambda x: get_year_month(x))
fw= fw.drop(['base_date'], axis=1, inplace=False)
card['year_month'] = card['base_date'].apply(lambda x: get_year_month(x))
card= card.drop(['base_date'], axis=1, inplace=False)
card.rename(columns={'cluster':'emd_cluster'}, inplace=True)

In [213]:
card_1 = card.groupby(['year_month','emd_nm','emd_cluster','card_cluster']).sum().reset_index()
card_1= card_1.drop(['Unnamed: 0','use_cnt'], axis=1, inplace=False)

In [214]:
fw_1 = fw.groupby(['emd_nm', 'year_month']).sum().reset_index()

In [216]:
df= pd.merge(fw_1, card_1, on=['year_month','emd_nm'])
df2= df.drop(['emd_nm','em_cnt','pay_amt'], axis=1)

In [220]:
df2['year_month'] = pd.to_datetime(df2['year_month'])
df2.set_index('year_month',inplace=True)

In [261]:
from statsmodels.tsa.stattools import coint
temp_cluster = []
temp_diff = []
temp_log = []
temp_log_diff = []

for i in range(1,10):
    temp = df2[df2['emd_cluster']==i]
    card_list = set(temp['card_cluster'])
    
    for j in card_list:
        temp2= temp[temp['card_cluster']==j]
        temp3 = pd.DataFrame()
        temp_diff_result= coint(temp2['use_amt'].diff().dropna(), temp2['em_g'].diff().dropna())
        temp_result = coint(np.log(temp2['use_amt']),np.log(temp2['em_g']))
        temp_result_diff = coint(np.log(temp2['use_amt']).diff().dropna(), np.log(temp2['em_g']).diff().dropna())
    
        print("diff")
        print("p-values = ", round(temp_diff_result[1],3), " : emd_cluster", i, ' card_cluster', j)
        print("log")
        print("p-values = ", round(temp_result[1],3), " : emd_cluster", i, ' card_cluster', j)
        print("log.diff")
        print("p-values = ", round(temp_result_diff[1],3), " : emd_cluster", i, ' card_cluster', j)
        temp_cluster.append(str(i) + '-' + str(j))
        
        if (temp_diff_result[1] < 0.05):
            temp_diff.append(temp_diff_result[1])
        else:
            temp_diff.append(np.nan)
        if (temp_result[1] < 0.05):
            temp_log.append(temp_result[1])
        else:
            temp_log.append(np.nan)
        if (temp_result_diff[1] < 0.05):
            temp_log_diff.append(temp_result_diff[1])
        else:
            temp_log_diff.append(np.nan)

diff
p-values =  0.0  : emd_cluster 1  card_cluster 1
log
p-values =  0.003  : emd_cluster 1  card_cluster 1
log.diff
p-values =  0.0  : emd_cluster 1  card_cluster 1
diff
p-values =  0.005  : emd_cluster 1  card_cluster 2
log
p-values =  0.001  : emd_cluster 1  card_cluster 2
log.diff
p-values =  0.003  : emd_cluster 1  card_cluster 2
diff
p-values =  0.0  : emd_cluster 1  card_cluster 3
log
p-values =  0.083  : emd_cluster 1  card_cluster 3
log.diff
p-values =  0.0  : emd_cluster 1  card_cluster 3
diff
p-values =  0.0  : emd_cluster 2  card_cluster 1
log
p-values =  0.004  : emd_cluster 2  card_cluster 1
log.diff
p-values =  0.0  : emd_cluster 2  card_cluster 1
diff
p-values =  0.0  : emd_cluster 2  card_cluster 2
log
p-values =  0.41  : emd_cluster 2  card_cluster 2
log.diff
p-values =  0.0  : emd_cluster 2  card_cluster 2
diff
p-values =  0.129  : emd_cluster 2  card_cluster 3
log
p-values =  0.0  : emd_cluster 2  card_cluster 3
log.diff
p-values =  0.013  : emd_cluster 2  card_clu

In [263]:
method_card = pd.DataFrame()
from statsmodels.tsa.stattools import coint
temp_cluster = []
temp_diff = []
temp_log = []
temp_log_diff = []
for i in range(1,10):
    temp = df2[df2['emd_cluster']==i]
    card_list = set(temp['card_cluster'])
    
    for j in card_list:
        temp2= temp[temp['card_cluster']==j]
        temp3 = pd.DataFrame()
        temp_diff_result= coint(temp2['use_amt'].diff().dropna(), temp2['em_g'].diff().dropna())
        temp_result = coint(np.log(temp2['use_amt']),np.log(temp2['em_g']))
        temp_result_diff = coint(np.log(temp2['use_amt']).diff().dropna(), np.log(temp2['em_g']).diff().dropna())
    
        print("diff")
        print("p-values = ", round(temp_diff_result[1],3), " : emd_cluster", i, ' card_cluster', j)
        print("log")
        print("p-values = ", round(temp_result[1],3), " : emd_cluster", i, ' card_cluster', j)
        print("log.diff")
        print("p-values = ", round(temp_result_diff[1],3), " : emd_cluster", i, ' card_cluster', j)
        temp_cluster.append(str(i) + '-' + str(j))
        temp_diff.append(temp_diff_result[1])
        temp_log.append(temp_result[1])
        temp_log_diff.append(temp_result_diff[1])
        
method_card['cluster'] = temp_cluster
method_card['diff'] = temp_diff
method_card['log'] = temp_log
method_card['log_diff'] = temp_log_diff
method_card['method'] = method_card[['diff', 'log', 'log_diff']].idxmin(axis=1)
method_card

diff
p-values =  0.0  : emd_cluster 1  card_cluster 1
log
p-values =  0.003  : emd_cluster 1  card_cluster 1
log.diff
p-values =  0.0  : emd_cluster 1  card_cluster 1
diff
p-values =  0.005  : emd_cluster 1  card_cluster 2
log
p-values =  0.001  : emd_cluster 1  card_cluster 2
log.diff
p-values =  0.003  : emd_cluster 1  card_cluster 2
diff
p-values =  0.0  : emd_cluster 1  card_cluster 3
log
p-values =  0.083  : emd_cluster 1  card_cluster 3
log.diff
p-values =  0.0  : emd_cluster 1  card_cluster 3
diff
p-values =  0.0  : emd_cluster 2  card_cluster 1
log
p-values =  0.004  : emd_cluster 2  card_cluster 1
log.diff
p-values =  0.0  : emd_cluster 2  card_cluster 1
diff
p-values =  0.0  : emd_cluster 2  card_cluster 2
log
p-values =  0.41  : emd_cluster 2  card_cluster 2
log.diff
p-values =  0.0  : emd_cluster 2  card_cluster 2
diff
p-values =  0.129  : emd_cluster 2  card_cluster 3
log
p-values =  0.0  : emd_cluster 2  card_cluster 3
log.diff
p-values =  0.013  : emd_cluster 2  card_clu

,cluster,diff,log,log_diff,method
0,1-1,5.278388e-05,3.368852e-03,2.880922e-05,log_diff
1,1-2,4.698784e-03,7.407682e-04,3.199905e-03,log
2,1-3,1.670749e-23,8.275606e-02,4.224563e-09,diff
3,2-1,7.820559e-07,3.547385e-03,9.509564e-07,diff
4,2-2,7.922428e-08,4.097415e-01,1.882422e-15,log_diff
5,2-3,1.292398e-01,1.034523e-06,1.314480e-02,log
6,3-1,1.123896e-02,1.939177e-02,2.776727e-25,log_diff
7,3-2,2.809855e-04,1.778620e-02,5.825173e-09,log_diff
8,3-3,5.437011e-04,1.371937e-03,2.247412e-29,log_diff
9,4-1,4.678108e-14,8.269221e-01,1.555362e-14,log_diff


In [264]:
method_card.to_csv("card_coint.csv", index=False)

In [230]:
from statsmodels.tsa.stattools import coint
for i in range(1,10):
    temp = df2[df2['emd_cluster']==i]
    card_list = set(temp['card_cluster'])
    for j in card_list:
        temp2= temp[temp['card_cluster']==j]
        temp3 = pd.DataFrame()
        temp3['use_amt_diff'] = temp2['use_amt'].diff().dropna()
        temp3['em_g_diff'] = temp2['em_g'].diff().dropna()
        _, p_value, _ = coint(temp3['use_amt_diff'], temp3['em_g_diff'])
        print('card cluster %d p-value' %(i), round(p_value,3))
    print('___________________________')

card cluster 1 p-value 0.0
card cluster 1 p-value 0.005
card cluster 1 p-value 0.0
___________________________
card cluster 2 p-value 0.0
card cluster 2 p-value 0.0
card cluster 2 p-value 0.129
___________________________
card cluster 3 p-value 0.011
card cluster 3 p-value 0.0
card cluster 3 p-value 0.001
___________________________
card cluster 4 p-value 0.0
card cluster 4 p-value 0.0
card cluster 4 p-value 0.0
___________________________
card cluster 5 p-value 0.0
card cluster 5 p-value 0.0
card cluster 5 p-value 0.0
___________________________
card cluster 6 p-value 0.0
card cluster 6 p-value 0.001
card cluster 6 p-value 0.0
___________________________
card cluster 7 p-value 0.0
card cluster 7 p-value 0.0
card cluster 7 p-value 0.0
___________________________
card cluster 8 p-value 0.0
card cluster 8 p-value 0.0
card cluster 8 p-value 0.0
___________________________
card cluster 9 p-value 0.0
card cluster 9 p-value 0.0
card cluster 9 p-value 0.0
___________________________


In [236]:
# 로그, 차분
from statsmodels.tsa.stattools import coint
for i in range(1,10):
    temp = df2[df2['emd_cluster']==i]
    card_list = set(temp['card_cluster'])
    for j in card_list:
        temp2= temp[temp['card_cluster']==j]
        temp3 = pd.DataFrame()
        temp3['use_amt_diff'] = np.log(temp2['use_amt']).diff().dropna()
        temp3['em_g_diff'] = np.log(temp2['em_g']).diff().dropna()
        _, p_value, _ = coint(temp3['use_amt_diff'], temp3['em_g_diff'])
        print('card cluster %d p-value' %(j), round(p_value,3))
    print('___________________________')

card cluster 1 p-value 0.0
card cluster 2 p-value 0.003
card cluster 3 p-value 0.0
___________________________
card cluster 1 p-value 0.0
card cluster 2 p-value 0.0
card cluster 3 p-value 0.013
___________________________
card cluster 1 p-value 0.0
card cluster 2 p-value 0.0
card cluster 3 p-value 0.0
___________________________
card cluster 1 p-value 0.0
card cluster 2 p-value 0.0
card cluster 3 p-value 0.0
___________________________
card cluster 1 p-value 0.0
card cluster 2 p-value 0.0
card cluster 3 p-value 0.0
___________________________
card cluster 1 p-value 0.0
card cluster 2 p-value 0.0
card cluster 3 p-value 0.0
___________________________
card cluster 1 p-value 0.0
card cluster 2 p-value 0.0
card cluster 3 p-value 0.0
___________________________
card cluster 1 p-value 0.0
card cluster 2 p-value 0.0
card cluster 3 p-value 0.0
___________________________
card cluster 1 p-value 0.0
card cluster 2 p-value 0.0
card cluster 3 p-value 0.0
___________________________


In [235]:
# 로그
from statsmodels.tsa.stattools import coint
for i in range(1,10):
    temp = df2[df2['emd_cluster']==i]
    card_list = set(temp['card_cluster'])
    for j in card_list:
        temp2= temp[temp['card_cluster']==j]
        temp3 = pd.DataFrame()
        temp3['use_amt_diff'] = np.log(temp2['use_amt'])
        temp3['em_g_diff'] = np.log(temp2['em_g'])
        _, p_value, _ = coint(temp3['use_amt_diff'], temp3['em_g_diff'])
        print('card cluster %d p-value' %(j), round(p_value,3))
    print('___________________________')

card cluster 1 p-value 0.003
card cluster 2 p-value 0.001
card cluster 3 p-value 0.083
___________________________
card cluster 1 p-value 0.004
card cluster 2 p-value 0.41
card cluster 3 p-value 0.0
___________________________
card cluster 1 p-value 0.019
card cluster 2 p-value 0.018
card cluster 3 p-value 0.001
___________________________
card cluster 1 p-value 0.827
card cluster 2 p-value 0.22
card cluster 3 p-value 0.485
___________________________
card cluster 1 p-value 0.783
card cluster 2 p-value 0.605
card cluster 3 p-value 0.0
___________________________
card cluster 1 p-value 0.703
card cluster 2 p-value 0.745
card cluster 3 p-value 0.33
___________________________
card cluster 1 p-value 0.983
card cluster 2 p-value 0.636
card cluster 3 p-value 0.028
___________________________
card cluster 1 p-value 0.325
card cluster 2 p-value 0.08
card cluster 3 p-value 0.059
___________________________
card cluster 1 p-value 0.032
card cluster 2 p-value 0.011
card cluster 3 p-value 0.003
_